In [11]:
from langgraph.graph import StateGraph,START,END
from pydantic import BaseModel,Field

In [12]:
class DiscountState(BaseModel):
    total: float = Field(..., ge=0, description="Total price of the order")
    discount_rate: float = Field(0.0, ge=0.0, le=1.0, description="Discount rate to be applied")
    final_price: float = Field(0.0,ge=0.0, description="Final price after discount")

In [ ]:
def apply_discount(state: DiscountState) :
    total = state.total   
    rate = state.discount_rate
    final = total * (1 - rate)
    return {'final_price': final}

In [14]:
def check_eligibility(state: DiscountState) :
    total = state.total
    
    if total > 1000:
        return {'discount_rate':0.20}
    elif total > 500:
        return {'discount_rate':0.10}
    else:
        return {'discount_rate':0.0}
        

In [15]:
def no_discount(state: DiscountState) :
    return {'final_price': state.total}

In [16]:
def router_discount(state: DiscountState) -> str:
    if state.discount_rate > 0:
        return 'apply_discount'
    else:
        return 'no_discount'
    

In [17]:
graph = StateGraph(DiscountState)

graph.add_node("check_eligibility",check_eligibility)

graph.add_node("apply_discount",apply_discount)

graph.add_node("no_discount",no_discount)


graph.add_edge( START,"check_eligibility" )

graph.add_conditional_edges("check_eligibility",router_discount, {
    "apply_discount":"apply_discount",
    "no_discount":"no_discount"
})

graph.add_edge("apply_discount", END)

graph.add_edge("no_discount", END)


In [18]:
workflow = graph.compile()

In [19]:
response = workflow.invoke({'total': -100})

ValidationError: 1 validation error for DiscountState
total
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=-100, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than_equal

In [ ]:
print(response)

{'total': 1200.0, 'discount_rate': 0.2, 'final_price': 960.0}


In [ ]:
response = workflow.invoke({'total': 600.0})

In [ ]:
print(response)

{'total': 600.0, 'discount_rate': 0.1, 'final_price': 540.0}


In [ ]:
response = workflow.invoke({'total': 400.0})

In [ ]:
print(response)

{'total': 400.0, 'discount_rate': 0.0, 'final_price': 400.0}
